In [1]:
import re
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import pandas as pd
import os
import re
import unicodedata
from pathlib import Path
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from datetime import datetime

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

print(os.getcwd())


C:\Users\steve\OneDrive - NOLA Business IT\source\repos\grocery-ml


In [2]:
class WinnDixieRecptParser:

    STORE_MANAGER_PATTERN = re.compile(r'store\s*manager\s*:\s*(?P<manager>.+)', re.IGNORECASE)
    CASHIER_PATTERN       = re.compile(r'your\s*cashier\s*:\s*(?P<cashier>.+)', re.IGNORECASE)
    HEADER_PATTERN = re.compile(r'^\s*Reg\s+You\s*Pay\s*$', re.IGNORECASE)
    STOP_PATTERN     = re.compile(r'\btotal\s*items\s*sold\b', re.IGNORECASE)
    COUNT_PATTERN = re.compile( r'\btotal\s*(?:number\s*of\s*)?items\s*sold\b[^0-9]*([0-9]+)',re.IGNORECASE)
    
    DATETIME_PATTERN = re.compile( r'(?P<date>\d{1,2}/\d{1,2}/\d{2,4})\s*(?:,?\s*at\s+)?(?P<time>\d{1,2}:\d{2}(?:\s*[AP]M)?)',re.IGNORECASE)

    ITEM_PATTERN = re.compile(
        r'^(?:QTY\s*(?P<qty>\d+)\s+)?'      # optional QTY
        r'(?P<item>.*?)(?=\s+\$)'           # item text up to first $
        r'\s+\$(?P<reg>\d+\.\d{2})'         # Reg price
        r'\s+\$(?P<pay>\d+\.\d{2})'         # You Pay price
        r'(?:\s+[A-Z])?\s*$',               # optional trailing flag like B
        re.IGNORECASE
    )


    #ef parse_file_to_rows(path: Path) -> List[Dict[str, Any]]:
    #    text = path.read_text(encoding='utf-8', errors='ignore')
    #    #print(text)
    #    result = parse_receipt_text(text)
    #    print(result)
    #    rows: List[Dict[str, Any]] = []
    #    for r in result['items']:
    #        rows.append({
    #            'source': path.name,
    #            'date': result['date'],
    #            'time': result['time'],
    #            'item': r['item'],
    #            'qty': r['qty'],
    #            'reg': r['reg'],
    #            'youPay': r['youPay'],
    #            'reportedItemsSold': result['reported'],
    #            'rowsMatchReported': result['validation']['rowsMatchReported'],
    #            'qtyMatchReported': result['validation']['qtyMatchReported']
    #       })
    #   return rows
    #########################################################################

    def parse(self, text: str) -> Dict[str, Any]:
        lines = [self.normalize_spaces(ln) for ln in text.splitlines() if ln.strip()]
    
        date, time = self.extract_datetime(lines)
        manager = self.extract_store_manager(lines)
        cashier = self.extract_cashier(lines)
    
        start = self.find_header_index(lines)
        if start is None:
            return {
                'items': [],
                'reported': None,
                'date': date,
                'time': time,
                'manager': manager,
                'cashier': cashier,
                'validation': {'rowsMatchReported': False, 'qtyMatchReported': False, 'rowsCount': 0, 'qtySum': 0}
            }

        items, reported = self.parse_items(lines, start)
        validation = self.validate_counts(items, reported)

        return {
            'items': items,
            'reported': reported,
            'date': date,
            'time': time,
            'manager': manager,
            'cashier': cashier,
            'validation': validation
        }

    #########################################################################

    def parse_items(self, lines, start_idx):
        items = []
        reported = None
        for ln in lines[start_idx:]:
            if self.clean_you_save(ln):
                continue
            if self.clean_you_save(ln) or self.clean_coupon(ln):
                continue    
            if not ln.strip():
                continue
    
            m_stop = self.COUNT_PATTERN.search(ln)
            if m_stop:
                reported = int(m_stop.group(1))
                break
    
            m = self.ITEM_PATTERN.match(ln)
            if m:
                qty = int(m.group('qty')) if m.group('qty') else 1
                items.append({
                    "item": m.group("item"),
                    "qty": qty,
                    "reg": float(m.group("reg")),
                    "youPay": float(m.group("pay")),
                })
        return items, reported
    
    #########################################################################
    
    def clean_coupon(self, line: str) -> bool:
        """Skip scanned coupon lines."""
        return re.search(r'^\s*CPN\s+SCANNED\s+COUPON', line, re.IGNORECASE) is not None
    #########################################################################
    
    def clean_you_save(self, line: str) -> bool:
        """Return True if line should be skipped because it starts with 'You save'."""
        return re.search(r'^\s*you\s*save\b', line, re.IGNORECASE) is not None
    #########################################################################
    
    #def clean_lines(text: str, exclude: Optional[List[re.Pattern]] = None) -> List[str]:
    #    if exclude is None:
    #        exclude = [
    #            re.compile(r'\b(subtotal|tax|change|thank\s*you|cash|card|tender)\b', re.IGNORECASE),
    #            re.compile(r'^\s*$')
    #        ]
    #    return [ln.strip() for ln in text.splitlines() if not any(p.search(ln) for p in exclude)]
    #########################################################################
    
    def normalize_spaces(self, s: str) -> str:
        s = unicodedata.normalize("NFKC", s)
        return re.sub(r'\s+', ' ', s.strip())
    #########################################################################
    
    def extract_datetime(self, lines):
        for ln in lines:
            m = self.DATETIME_PATTERN.search(ln)
            if m:
                return m.group('date'), m.group('time')
        return None, None
    #########################################################################
        
    def find_header_index(self, lines: List[str]) -> Optional[int]:
        for i, ln in enumerate(lines):
            if self.HEADER_PATTERN.search(ln):
                return i + 1
        return None
    #########################################################################
    
    def validate_counts(self, items: List[Dict[str, Any]], reported: Optional[int]) -> Dict[str, Any]:
        rows_count = len(items)
        qty_sum = sum(r['qty'] for r in items)
        return {
            'rowsMatchReported': (reported is not None and rows_count == reported),
            'qtyMatchReported': (reported is not None and qty_sum == reported),
            'rowsCount': rows_count,
            'qtySum': qty_sum
        }
    #########################################################################
    
    def extract_store_manager(self, lines: List[str]) -> Optional[str]:
        for ln in lines:
            m = self.STORE_MANAGER_PATTERN.search(ln)
            if m:
                return m.group('manager').strip()
        return None
    #########################################################################
    
    def extract_cashier(self, lines: List[str]) -> Optional[str]:
        for ln in lines:
            m = self.CASHIER_PATTERN.search(ln)
            if m:
                return m.group('cashier').strip()
        return None
     #########################################################################


In [3]:

rows = []

recptParser  = WinnDixieRecptParser();

for p in Path("StevePhone2/pdf/text").glob("*.txt"):
    result = recptParser.parse(p.read_text(encoding="utf-8", errors="ignore"))
    for r in result["items"]:
        rows.append({
            "source": p.name,
            "date": result["date"],
            "time": result["time"],
            "manager": result["manager"],
            "cashier": result["cashier"],
            "item": r["item"],
            "qty": r["qty"],
            "reg": r["reg"],
            "youPay": r["youPay"],
            "reportedItemsSold": result["reported"],
            #"rowsMatchReported": result["validation"]["rowsMatchReported"],
            "qtyMatchReported": result["validation"]["qtyMatchReported"],
        })

df = pd.DataFrame(rows)


# new features

In [4]:
# 1) build a proper timestamp
df["dateTime"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str), errors="coerce")

df = df.drop(columns=["date", "time"])


dt = df["dateTime"]

df["year"]    = dt.dt.year
df["month"]   = dt.dt.month
df["day"]     = dt.dt.day
df["hour"]    = dt.dt.hour
df["minute"]  = dt.dt.minute
df["dow"]     = dt.dt.dayofweek
df["doy"]     = dt.dt.dayofyear
df["quarter"] = dt.dt.quarter



C:\Users\steve\AppData\Local\Temp\ipykernel_14900\3748699370.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["dateTime"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str), errors="coerce")


In [5]:
# discount 
df["discount"] = df["reg"] - df["youPay"]

# total discount per trip
df["totalDiscountPerTrip"] = df.groupby("dateTime")["discount"].transform("sum")

df["totalTripSpend"] = df.groupby("dateTime")["youPay"].transform("sum")

,source,manager,cashier,item,qty,reg,youPay,reportedItemsSold,qtyMatchReported,dateTime,year,month,day,hour,minute,dow,doy,quarter,discount,totalDiscountPerTrip,totalTripSpend
0,IMG_9655.txt,PAUL,STEPHEN,k&l Canned Beans,1,1.39,0.99,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.40,0.90,12.16
1,IMG_9655.txt,PAUL,STEPHEN,Blue Runner Beans,1,3.79,3.79,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.00,0.90,12.16
2,IMG_9655.txt,PAUL,STEPHEN,Manda Sausage,1,5.69,5.19,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.50,0.90,12.16
3,IMG_9655.txt,PAUL,STEPHEN,SEG BOIL-N-BAG,1,2.19,2.19,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.00,0.90,12.16
4,IMG_9656.txt,PAUL,ACM LANE_73,Hugbi Pies,2,5.38,5.38,18,True,2025-11-04 14:00:00,2025,11,4,14,0,1,308,4,0.00,52.01,102.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,IMG_9764.txt,None,STEPHEN,M&M's Candies,1,1.79,1.79,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,0.00,2.26,21.26
764,IMG_9764.txt,None,STEPHEN,Mandarins,1,6.99,5.99,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,1.00,2.26,21.26
765,IMG_9764.txt,None,STEPHEN,Sparkling Ice Wtr,1,1.29,1.00,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,0.29,2.26,21.26
766,IMG_9764.txt,None,STEPHEN,Sparkling Ice Wtr,1,1.29,1.00,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,0.29,2.26,21.26


In [6]:
df.to_csv("receipts.csv", index=False, encoding="utf-8")

# DF CHECK

In [7]:
# Read CSV
df_check = pd.read_csv("ListCheck.csv")
df_check.columns = df_check.columns.str.strip().str.lower()

df_check


,datetime,location,cost,itemcount
0,11/05/2025 5:59 PM,Winn-Dixie 2104 Williams Blvd,$12.59,4 Items
1,11/04/2025 2:00 PM,Winn-Dixie 2104 Williams Blvd,$106.36,18 Items
2,10/30/2025 8:45 PM,Winn-Dixie 2104 Williams Blvd,$15.89,6 Items
3,10/27/2025 8:36 PM,Winn-Dixie 2104 Williams Blvd,$8.88,2 Items
4,10/25/2025 11:01 AM,Winn-Dixie 2104 Williams Blvd,$56.73,15 Items
...,...,...,...,...
130,11/26/2024 7:42 PM,Winn-Dixie 2104 Williams Blvd,$145.07,32 Items
131,11/24/2024 3:40 PM,Winn-Dixie 2104 Williams Blvd,$57.03,16 Items
132,11/22/2024 9:15 PM,Winn-Dixie 2104 Williams Blvd,$8.27,2 Items
133,11/17/2024 10:35 AM,Winn-Dixie 2104 Williams Blvd,$85.44,27 Items


In [8]:
df_check["datetime"] = pd.to_datetime(df_check["datetime"], errors="coerce")

C:\Users\steve\AppData\Local\Temp\ipykernel_14900\1989889571.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_check["datetime"] = pd.to_datetime(df_check["datetime"], errors="coerce")


In [9]:
df_check["cost"] = df_check["cost"].astype(str).str.replace(r"[\$,]", "", regex=True).astype(float)
df_check["itemcount"] = df_check["itemcount"].astype(str).str.extract(r"(\d+)").astype(int)
                                                                        
df_check = df_check.drop(columns="Location", errors="ignore")

# Compare with df_train trips by datetime and item count
missing = df_check[~df_check["datetime"].isin(df["dateTime"])]

missing

,datetime,location,cost,itemcount
15,2025-10-07 18:05:00,Winn-Dixie 2104 Williams Blvd,37.94,10
18,2025-09-28 17:02:00,Winn-Dixie 2104 Williams Blvd,63.15,21
21,2025-09-21 15:11:00,Winn-Dixie 2104 Williams Blvd,80.21,27
32,2025-08-25 19:11:00,Winn-Dixie 2104 Williams Blvd,95.20,25
41,2025-08-06 15:54:00,Winn-Dixie 2104 Williams Blvd,81.26,25
47,2025-07-24 14:47:00,Winn-Dixie 2104 Williams Blvd,84.64,25
54,2025-07-06 19:26:00,Winn-Dixie 2104 Williams Blvd,73.52,27
59,2025-06-14 18:03:00,Winn-Dixie 2104 Williams Blvd,161.20,35
60,2025-06-14 18:01:00,Winn-Dixie 2104 Williams Blvd,22.12,3
62,2025-06-08 20:18:00,Winn-Dixie 2104 Williams Blvd,108.31,25


In [10]:

df["totalTripSpend"] = df["totalTripSpend"].astype(float)

# merge for comparison
merged = pd.merge(
    df_check,
    df[["dateTime", "totalTripSpend"]],
    how="left",
    left_on="datetime",
    right_on="dateTime",
    indicator=True
)

# find rows missing or with cost mismatch
missing = merged[merged["_merge"] == "left_only"]
mismatch = merged[abs(merged["cost"] - merged["totalTripSpend"]) > 0.01]

merged


,datetime,location,cost,itemcount,dateTime,totalTripSpend,_merge
0,2025-11-05 17:59:00,Winn-Dixie 2104 Williams Blvd,12.59,4,2025-11-05 17:59:00,12.16,both
1,2025-11-05 17:59:00,Winn-Dixie 2104 Williams Blvd,12.59,4,2025-11-05 17:59:00,12.16,both
2,2025-11-05 17:59:00,Winn-Dixie 2104 Williams Blvd,12.59,4,2025-11-05 17:59:00,12.16,both
3,2025-11-05 17:59:00,Winn-Dixie 2104 Williams Blvd,12.59,4,2025-11-05 17:59:00,12.16,both
4,2025-11-04 14:00:00,Winn-Dixie 2104 Williams Blvd,106.36,18,2025-11-04 14:00:00,102.76,both
...,...,...,...,...,...,...,...
809,2024-11-15 20:13:00,Winn-Dixie 2104 Williams Blvd,22.00,8,2024-11-15 20:13:00,21.26,both
810,2024-11-15 20:13:00,Winn-Dixie 2104 Williams Blvd,22.00,8,2024-11-15 20:13:00,21.26,both
811,2024-11-15 20:13:00,Winn-Dixie 2104 Williams Blvd,22.00,8,2024-11-15 20:13:00,21.26,both
812,2024-11-15 20:13:00,Winn-Dixie 2104 Williams Blvd,22.00,8,2024-11-15 20:13:00,21.26,both


# DF TRAIN

In [11]:
df_train = df.copy()

df_train


,source,manager,cashier,item,qty,reg,youPay,reportedItemsSold,qtyMatchReported,dateTime,year,month,day,hour,minute,dow,doy,quarter,discount,totalDiscountPerTrip,totalTripSpend
0,IMG_9655.txt,PAUL,STEPHEN,k&l Canned Beans,1,1.39,0.99,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.40,0.90,12.16
1,IMG_9655.txt,PAUL,STEPHEN,Blue Runner Beans,1,3.79,3.79,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.00,0.90,12.16
2,IMG_9655.txt,PAUL,STEPHEN,Manda Sausage,1,5.69,5.19,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.50,0.90,12.16
3,IMG_9655.txt,PAUL,STEPHEN,SEG BOIL-N-BAG,1,2.19,2.19,4,True,2025-11-05 17:59:00,2025,11,5,17,59,2,309,4,0.00,0.90,12.16
4,IMG_9656.txt,PAUL,ACM LANE_73,Hugbi Pies,2,5.38,5.38,18,True,2025-11-04 14:00:00,2025,11,4,14,0,1,308,4,0.00,52.01,102.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,IMG_9764.txt,None,STEPHEN,M&M's Candies,1,1.79,1.79,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,0.00,2.26,21.26
764,IMG_9764.txt,None,STEPHEN,Mandarins,1,6.99,5.99,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,1.00,2.26,21.26
765,IMG_9764.txt,None,STEPHEN,Sparkling Ice Wtr,1,1.29,1.00,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,0.29,2.26,21.26
766,IMG_9764.txt,None,STEPHEN,Sparkling Ice Wtr,1,1.29,1.00,8,True,2024-11-15 20:13:00,2024,11,15,20,13,4,320,4,0.29,2.26,21.26


In [12]:


# assume df_train already has year, month, day, hour, minute, dow, doy, quarter
df_train["hour_sin"] = np.sin(2 * np.pi * df_train["hour"] / 24)
df_train["hour_cos"] = np.cos(2 * np.pi * df_train["hour"] / 24)

df_train["minute_sin"] = np.sin(2 * np.pi * df_train["minute"] / 60)
df_train["minute_cos"] = np.cos(2 * np.pi * df_train["minute"] / 60)

df_train["dow_sin"] = np.sin(2 * np.pi * df_train["dow"] / 7)
df_train["dow_cos"] = np.cos(2 * np.pi * df_train["dow"] / 7)

df_train["month_sin"] = np.sin(2 * np.pi * df_train["month"] / 12)
df_train["month_cos"] = np.cos(2 * np.pi * df_train["month"] / 12)

df_train["doy_sin"] = np.sin(2 * np.pi * df_train["doy"] / 365)
df_train["doy_cos"] = np.cos(2 * np.pi * df_train["doy"] / 365)

df_train["quarter_sin"] = np.sin(2 * np.pi * df_train["quarter"] / 4)
df_train["quarter_cos"] = np.cos(2 * np.pi * df_train["quarter"] / 4)

df_train = df_train.drop(columns=["year", "month", "day", "hour", "minute", "dow", "doy", "quarter"])




In [13]:

scaler_reg = StandardScaler()
df_train["regScaled"] = scaler_reg.fit_transform(df_train[["reg"]].fillna(0.0))
df_train = df_train.drop(columns="reg", errors="ignore")

scaler_youPay = StandardScaler()
df_train["youPayScaled"] = scaler_youPay.fit_transform(df_train[["youPay"]].fillna(0.0))
df_train = df_train.drop(columns="youPay", errors="ignore")

scaler_discount = StandardScaler()
df_train["discountScaled"] = scaler_discount.fit_transform(df_train[["discount"]].fillna(0.0))
df_train = df_train.drop(columns="discount", errors="ignore")

scaler_totalDiscountPerTrip = StandardScaler()
df_train["totalDiscountPerTripScaled"] = scaler_totalDiscountPerTrip.fit_transform(df_train[["totalDiscountPerTrip"]].fillna(0.0))
df_train = df_train.drop(columns="totalDiscountPerTrip", errors="ignore")

scaler_totalTripSpend = StandardScaler()
df_train["totalTripSpendScaled"] = scaler_totalTripSpend.fit_transform(df_train[["totalTripSpend"]].fillna(0.0))
df_train = df_train.drop(columns="totalTripSpend", errors="ignore")

scaler_reported = StandardScaler()
df_train["reportedItemsSoldScaled"] = scaler_reported.fit_transform(df_train[["reportedItemsSold"]].fillna(0.0))
df_train = df_train.drop(columns="reportedItemsSold", errors="ignore")

df_train

,source,manager,cashier,item,qty,qtyMatchReported,dateTime,hour_sin,hour_cos,minute_sin,minute_cos,dow_sin,dow_cos,month_sin,month_cos,doy_sin,doy_cos,quarter_sin,quarter_cos,regScaled,youPayScaled,discountScaled,totalDiscountPerTripScaled,totalTripSpendScaled,reportedItemsSoldScaled
0,IMG_9655.txt,PAUL,STEPHEN,k&l Canned Beans,1,True,2025-11-05 17:59:00,-0.965926,-0.258819,-0.104528,0.994522,0.974928,-0.222521,-0.5,0.866025,-0.821477,0.570242,-2.449294e-16,1.0,-0.509282,-0.651729,-0.235017,-0.905595,-1.468750,-1.470757
1,IMG_9655.txt,PAUL,STEPHEN,Blue Runner Beans,1,True,2025-11-05 17:59:00,-0.965926,-0.258819,-0.104528,0.994522,0.974928,-0.222521,-0.5,0.866025,-0.821477,0.570242,-2.449294e-16,1.0,-0.275198,-0.223335,-0.332757,-0.905595,-1.468750,-1.470757
2,IMG_9655.txt,PAUL,STEPHEN,Manda Sausage,1,True,2025-11-05 17:59:00,-0.965926,-0.258819,-0.104528,0.994522,0.974928,-0.222521,-0.5,0.866025,-0.821477,0.570242,-2.449294e-16,1.0,-0.089882,-0.009138,-0.210582,-0.905595,-1.468750,-1.470757
3,IMG_9655.txt,PAUL,STEPHEN,SEG BOIL-N-BAG,1,True,2025-11-05 17:59:00,-0.965926,-0.258819,-0.104528,0.994522,0.974928,-0.222521,-0.5,0.866025,-0.821477,0.570242,-2.449294e-16,1.0,-0.431254,-0.468132,-0.332757,-0.905595,-1.468750,-1.470757
4,IMG_9656.txt,PAUL,ACM LANE_73,Hugbi Pies,2,True,2025-11-04 14:00:00,-0.500000,-0.866025,0.000000,1.000000,0.781831,0.623490,-0.5,0.866025,-0.831171,0.556017,-2.449294e-16,1.0,-0.120118,0.019932,-0.332757,2.988070,2.061734,0.945743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,IMG_9764.txt,None,STEPHEN,M&M's Candies,1,True,2024-11-15 20:13:00,-0.866025,0.500000,0.978148,0.207912,-0.433884,-0.900969,-0.5,0.866025,-0.699458,0.714673,-2.449294e-16,1.0,-0.470268,-0.529331,-0.332757,-0.801988,-1.114143,-0.780328
764,IMG_9764.txt,None,STEPHEN,Mandarins,1,True,2024-11-15 20:13:00,-0.866025,0.500000,0.978148,0.207912,-0.433884,-0.900969,-0.5,0.866025,-0.699458,0.714673,-2.449294e-16,1.0,0.036913,0.113260,-0.088408,-0.801988,-1.114143,-0.780328
765,IMG_9764.txt,None,STEPHEN,Sparkling Ice Wtr,1,True,2024-11-15 20:13:00,-0.866025,0.500000,0.978148,0.207912,-0.433884,-0.900969,-0.5,0.866025,-0.699458,0.714673,-2.449294e-16,1.0,-0.519035,-0.650199,-0.261896,-0.801988,-1.114143,-0.780328
766,IMG_9764.txt,None,STEPHEN,Sparkling Ice Wtr,1,True,2024-11-15 20:13:00,-0.866025,0.500000,0.978148,0.207912,-0.433884,-0.900969,-0.5,0.866025,-0.699458,0.714673,-2.449294e-16,1.0,-0.519035,-0.650199,-0.261896,-0.801988,-1.114143,-0.780328


In [14]:

#le_manager = LabelEncoder()
#df_train["managerEncoded"] = le_manager.fit_transform(df_train["manager"].astype(str))
#df_train = df_train.drop(columns="manager", errors="ignore")

#le_cashier = LabelEncoder()
#df_train["cashierEncoded"] = le_cashier.fit_transform(df_train["cashier"].astype(str))
#df_train = df_train.drop(columns="cashier", errors="ignore")

#le_item = LabelEncoder()
#df_train["itemEncoded"] = le_item.fit_transform(df_train["item"].astype(str))
#df_train = df_train.drop(columns="item", errors="ignore")


#le_source = LabelEncoder()
#df_train["sourceEncoded"] = le_source.fit_transform(df_train["source"].astype(str))
#df_train = df_train.drop(columns="source", errors="ignore")


In [15]:
df_train.to_csv("df_train.csv", index=False, encoding="utf-8")

print("before")
print(df_train.columns.tolist())

cols = ['source', 'manager', 'cashier', 'item', 'qty', 'qtyMatchReported', 'dateTime', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos', 'quarter_sin', 'quarter_cos', 'regScaled', 'youPayScaled', 'discountScaled', 'totalDiscountPerTripScaled', 'totalTripSpendScaled', 'reportedItemsSoldScaled']
drop_cols = ['qty', 'qtyMatchReported',  'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos', 'quarter_sin', 'quarter_cos', 'regScaled', 'discountScaled', 'totalDiscountPerTripScaled', 'totalTripSpendScaled', 'reportedItemsSoldScaled']


#drop_cols = [ 'item', 'qty', 'qtyMatchReported', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos', 'quarter_sin', 'quarter_cos', 'regScaled', 'discountScaled', 'totalDiscountPerTripScaled', 'totalTripSpendScaled', 'reportedItemsSoldScaled']

df_train.drop(columns=drop_cols, inplace=True)

print("after")
print(df_train.columns.tolist())



before
['source', 'manager', 'cashier', 'item', 'qty', 'qtyMatchReported', 'dateTime', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos', 'quarter_sin', 'quarter_cos', 'regScaled', 'youPayScaled', 'discountScaled', 'totalDiscountPerTripScaled', 'totalTripSpendScaled', 'reportedItemsSoldScaled']
after
['source', 'manager', 'cashier', 'item', 'dateTime', 'youPayScaled']


# MODELs !!!

## Kmeans

In [16]:

numOfClusters = 4

# 1. Select numeric features
X = df_train.select_dtypes(include=[float, int])

# 2. Apply PCA (optional dimensionality reduction)
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X)

# 3. Fit KMeans and assign clusters

kmeans = KMeans(n_clusters=numOfClusters, random_state=42)
df_train["cluster"] = kmeans.fit_predict(X_pca)


### KMEANS ANALYZE

In [17]:
#plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df_train["cluster"], s=10, alpha=0.6)
#plt.xlabel("PCA1")
#plt.ylabel("PCA2")
#plt.title("K-Means Clusters")
#plt.show()

In [18]:
# assume df and df_train share same index
df_clusters = df.copy()
df_clusters['cluster'] = df_train['cluster']

# add per-cluster sample count
df_clusters['sampleQty'] = df_clusters.groupby('cluster')['cluster'].transform('count')

In [19]:
df_clusters['generated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_clusters.to_csv("clusters_full.csv", index=False)
print("Saved clusters_full.csv")


Saved clusters_full.csv


# Max Cluster

In [ ]:
max_clusters = 500           # user-defined

# 1. Select numeric columns from df_train
X = df_train.select_dtypes(include=["number"]).fillna(0.0)

# 2. Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Reduce dimensions with PCA (keep 95% variance)
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# 4. Define limits for cluster testing
max_allowed = X_pca.shape[0] - 1  # dataset cap
max_k = min(max_clusters, max_allowed)

# 5. Loop through cluster counts and calculate silhouette scores
results = []
for k in range(2, max_k + 1):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_pca)
    score = silhouette_score(X_pca, labels)
    results.append((k, score))
    #$print(f"k={k}, silhouette={score:.3f}")

# 6. Collect results and plot
scores_df = pd.DataFrame(results, columns=["k", "silhouette"])

plt.plot(scores_df["k"], scores_df["silhouette"], marker="o")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette vs Cluster Count")
plt.show()

C:\Users\steve\.conda\envs\intercard-ml\Lib\site-packages\sklearn\base.py:1365: ConvergenceWarning: Number of distinct clusters (193) found smaller than n_clusters (194). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
C:\Users\steve\.conda\envs\intercard-ml\Lib\site-packages\sklearn\base.py:1365: ConvergenceWarning: Number of distinct clusters (193) found smaller than n_clusters (195). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
C:\Users\steve\.conda\envs\intercard-ml\Lib\site-packages\sklearn\base.py:1365: ConvergenceWarning: Number of distinct clusters (193) found smaller than n_clusters (196). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
C:\Users\steve\.conda\envs\intercard-ml\Lib\site-packages\sklearn\base.py:1365: ConvergenceWarning: Number of distinct clusters (193) found smaller than n_clusters (197). Possibly due to duplicate points in X.
  return fit_met

In [ ]:
df_train["cluster"].value_counts()


In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df_train["cluster"], s=10, alpha=0.6)
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.title("K-Means Clusters")
plt.show()

In [ ]:
## scale features
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(df_train.select_dtypes(include=['number']))

# cluster
#dbscan = DBSCAN(eps=0.5, min_samples=5)
#labels = dbscan.fit_predict(X_scaled)

#df_train['cluster'] = labels